<a href="https://colab.research.google.com/github/tjwikki12/Ideathon/blob/main/Ideathon_web_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from time import sleep
import requests
from google.colab import files
import pandas as pd
from bs4 import BeautifulSoup

# Helper function to extract data from BeautifulSoup objects
def soup2list(src, list_, attr=None):
    if attr:
        for val in src:
            list_.append(val[attr])
    else:
        for val in src:
            list_.append(val.get_text(strip=True))  # Use strip=True to clean the text

# Initialize lists for storing extracted data
users = []
ratings = []
locations = []
dates = []
reviews = []
reviews_highlight = []  # To store the review titles/highlights
reviews_date = []  # To store the date of experience

from_page = 1
to_page = 2000
company = 'www.lebara.com'

# Loop over each page from 'from_page' to 'to_page'
for i in range(from_page, to_page + 1):

    result = requests.get(fr"https://www.trustpilot.com/review/{company}?page={i}")
    soup = BeautifulSoup(result.content, 'html.parser')

    # Extracting review content (review text)
    review_elements = soup.find_all('div', {'class': 'styles_reviewContent__0Q2Tg'})
    for review in review_elements:
        # Extract review highlight/title
        highlight = review.find('h2', {'class': 'typography_heading-s__f7029 typography_appearance-default__AAY17'})
        if highlight:
            reviews_highlight.append(highlight.get_text(strip=True))  # Extract and clean the title

        # Extract review text
        review_text = review.find('p', {'class': 'typography_body-l__KUYFJ typography_appearance-default__AAY17 typography_color-black__5LYEn'})
        if review_text:
            reviews.append(review_text.get_text(strip=True))  # Extract review content

        # Extract the date of experience
        date_element = review.find('p', {'class': 'typography_body-m__xgxZ_ typography_appearance-default__AAY17'})
        if date_element:
            # Clean the text to get the actual date of experience
            date_text = date_element.get_text(strip=True).replace("Date of experience:", "").strip()
            reviews_date.append(date_text)

    # Extract user names
    soup2list(soup.find_all('span', {'class': 'typography_heading-xxs__QKBS8 typography_appearance-default__AAY17'}), users)

    # Extract user locations
    soup2list(soup.find_all('div', {'class': 'typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_detailsIcon__Fo_ua'}), locations)

    # Extract review dates
    soup2list(soup.find_all('div', {'class': 'styles_reviewHeader__iU9Px'}), dates)

    # Extract ratings
    soup2list(soup.find_all('div', {'class': 'styles_reviewHeader__iU9Px'}), ratings, attr='data-service-review-rating')




In [ ]:
# need to ensure that all lists are of the same length
max_length = max(len(users), len(locations), len(dates), len(ratings), len(reviews), len(reviews_highlight), len(reviews_date))

# Fill the shorter lists with None to make all lists the same length
users.extend([None] * (max_length - len(users)))
locations.extend([None] * (max_length - len(locations)))
dates.extend([None] * (max_length - len(dates)))
ratings.extend([None] * (max_length - len(ratings)))
reviews.extend([None] * (max_length - len(reviews)))
reviews_highlight.extend([None] * (max_length - len(reviews_highlight)))
reviews_date.extend([None] * (max_length - len(reviews_date)))

# Create a DataFrame with the lists
df = pd.DataFrame({
    "User": users,
    "Location": locations,
    "Review Date": dates,
    "Rating": ratings,
    "Review Highlight": reviews_highlight,
    "Review": reviews,
    "Date of Experience": reviews_date
})

# Display DataFrame
df.head()

,User,Location,Review Date,Rating,Review Highlight,Review,Date of Experience
0,Jill Khoda,GB,11 hours ago,5,Karan is excellent.,Karan is excellent.. he resolved my issue very...,"December 04, 2024"
1,Grace,GB,12 hours ago,5,I was so worried about a debit using my…,I was so worried about a debit using my friend...,"December 04, 2024"
2,Ben Thomas,GB,14 hours ago,5,Lost SIM help,Lost SIM helpI was helped by Raunak who quickl...,"December 04, 2024"
3,JONATHAN CULMER,GB,2 days ago,5,Sim Only Savings,I switched over to Lebara as i saw their amazi...,"December 02, 2024"
4,MN,GB,2 days ago,5,Excellent service from Lebara!,Minimum delay in getting to speak with Lebara ...,"December 03, 2024"


In [ ]:
df.to_csv('reviews.csv', index=False)

In [ ]:
files.download('reviews.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>